<a href="https://colab.research.google.com/github/ZeleOeO/Artificial-Intelligence-Projects/blob/main/Gutenberg_Book_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First read the csv file from the place where I kept it

In [170]:
import pandas as pd

df = pd.read_csv("/content/gutenberg_metadata.csv")

print(df.shape)
df.head()

(15331, 4)


,Title,Author,Link,Bookshelf
0,The Extermination of the American Bison,William T. Hornaday,http://www.gutenberg.org/ebooks/17748,Animal
1,Deadfalls and Snares,A. R. Harding,http://www.gutenberg.org/ebooks/34110,Animal
2,Artistic Anatomy of Animals,Édouard Cuyer,http://www.gutenberg.org/ebooks/38315,Animal
3,"Birds, Illustrated","Color Photography, Vol. 1, No. 1 Various",http://www.gutenberg.org/ebooks/30221,Animal
4,On Snake-Poison: Its Action and Its Antidote,A. Mueller,http://www.gutenberg.org/ebooks/32947,Animal


Remove all NA stuff

In [171]:
#Removed the 2 irrelevant coumns
df = df.drop(
    labels=["Author", "Link"],
    axis=1
)
#Removed any row with an empty cell in it, with our new dataset
df=df.dropna()

#Removedthe category named 6, cause it doesn't make any sense...
df=df[df.Bookshelf!="6"]

print(df.shape)
df.head()

(2707, 2)


,Title,Bookshelf
0,The Extermination of the American Bison,Animal
1,Deadfalls and Snares,Animal
2,Artistic Anatomy of Animals,Animal
3,"Birds, Illustrated",Animal
4,On Snake-Poison: Its Action and Its Antidote,Animal


Numerization of the classifications

In [172]:
#Turned it into a set so that each element won't be duplicated
classes = set(df.Bookshelf)

#Created a dictionary to use to map each element to the other
mapping = {}

#Looped through the set and added each element with it's index as it's numeric substitute/value
for index, value in enumerate(classes):
  mapping[value]=index

#Created a new column in the dataframe with the numeric forms
df["label_num"] = df.Bookshelf.map(
    mapping
)
df

,Title,Bookshelf,label_num
0,The Extermination of the American Bison,Animal,104
1,Deadfalls and Snares,Animal,104
2,Artistic Anatomy of Animals,Animal,104
3,"Birds, Illustrated",Animal,104
4,On Snake-Poison: Its Action and Its Antidote,Animal,104
...,...,...,...
2727,Campaign Pictures of the War in South Africa (...,Boer,10
2728,Lord Milner's Work in South Africa,Boer,10
2729,The Relief of Mafeking,Boer,10
2730,The Boer in Peace and War,Boer,10


Preprocessing Function

In [173]:
import spacy

import spacy.cli

spacy.cli.download("en_core_web_lg")


#using the large model cause it will be more accurate
nlp = spacy.load("en_core_web_lg")

#Creating the function
def preprocess(text):
  doc = nlp(text)

  filtered_tokens=[]

  #loops through the sentence/doc
  for token in doc:
    #checks if it's a stopword or punctuation and "removes it"
    if token.is_stop or token.is_punct:
      continue
    #lemmatizes the rest of the tokens  
    filtered_tokens.append(token.lemma_)
  
  #returns the string form
  return " ".join(filtered_tokens)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


Create preprocessed column

In [174]:
#Convert the preprocesseed text to vector form in 300 by 300 form
df["text"] = df.Title.apply(preprocess).apply(lambda x: nlp(x).vector)

In [175]:
from sklearn.model_selection import train_test_split

#this is going to split the data into training and test data
#df.text would be our x
#df.label_num would be our y
#test size is the percentage of our data that would be used for testing, which is 20%
#random_state shuffles it, 2023 is just a random key
x_train, x_test, y_train, y_test = train_test_split(
    df.text,
    df.label_num,
    test_size = 0.2,
    random_state = 2022
)

We convert the x data to positive, cause it doens't train negatives


In [176]:
# Convert to positive
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()

train_scaler = scaler.fit_transform(np.stack(x_train))
test_scaler = scaler.fit_transform(np.stack(x_test))

In [177]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

from sklearn.metrics import classification_report

kn = KNeighborsClassifier()
kn.fit(train_scaler, y_train)

KNeighborsClassifier()

In [178]:
y_pred = kn.predict(test_scaler)
test_scaler.shape

(542, 300)

In [179]:
print(classification_report(np.stack(y_test), y_pred))

              precision    recall  f1-score   support

           0       0.30      0.64      0.41        11
           1       0.23      0.60      0.33         5
           2       0.00      0.00      0.00         0
           3       0.40      1.00      0.57         4
           4       0.11      0.25      0.15         4
           5       0.50      0.43      0.46         7
           6       0.00      0.00      0.00         1
           7       0.38      0.50      0.43         6
           8       0.33      0.50      0.40         8
           9       0.00      0.00      0.00         0
          10       0.33      0.80      0.47         5
          11       0.62      0.81      0.70        16
          12       0.40      1.00      0.57         2
          13       0.08      0.33      0.13         3
          14       0.33      0.33      0.33         3
          15       0.42      0.71      0.53         7
          16       0.50      1.00      0.67         1
          17       0.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1

In [180]:
#creating a function that converts it to the vector form the model was trained in
def sentence_creator(sentence):
  return np.stack(nlp(preprocess(sentence)).vector)

# creating a function to tell
def check_output(output):
  for i in output:
    for k,v in mapping.items():
      if i == v:
        print(k)

In [181]:
np.reshape(sentence_creator("Book about the animal kingdom"), (-1, 300))


array([[-0.48847   , -2.439967  , -1.8147033 , -3.4955997 ,  2.0361335 ,
        -1.5959567 ,  1.775424  ,  1.68398   , -1.3644567 , -0.2738067 ,
         4.820433  ,  0.3203967 , -4.42003   ,  2.4131935 ,  1.5991668 ,
        -1.0478367 ,  4.741     , -0.18477671, -0.76379997, -0.4917333 ,
        -2.8702335 ,  1.5681067 , -3.4155667 , -0.16772993, -1.8643332 ,
        -1.8971367 , -2.784753  , -0.41466668, -1.4092535 ,  3.1612365 ,
        -0.8448667 , -0.00753339,  0.22153334,  0.17293327, -1.48387   ,
        -2.760193  ,  0.9833    ,  2.0511367 ,  0.30457667,  0.42663327,
        -1.6830233 ,  1.1222132 , -0.6413767 ,  4.576433  ,  2.1988335 ,
         1.8362502 ,  2.3335836 ,  0.4071034 ,  0.12576668, -1.7294668 ,
        -1.3224334 ,  2.7850666 ,  1.5545901 , -1.9278132 , -4.6889    ,
         0.80973333, -1.3170866 , -0.9412624 ,  3.1314068 ,  2.3527067 ,
         0.49590334,  1.5215956 , -0.15670013,  0.0825274 ,  0.6771467 ,
         0.39694   , -3.7889998 , -5.2422996 ,  0.9

In [182]:
sent = "Arts and craft"
sent = np.reshape(sentence_creator(sent), (-1, 300))
pred = kn.predict(sent)
check_output(pred)

Art
